### 1. Requirement

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

A_to_au_conversion = 1.8897259885789

### 2. Define Function

In [3]:
def calc_adapt_vqe_LiH(r, E, active_electrons=2, active_orbitals=5):
    symbols = ["Li", "H"]
    geometry = np.array([0.0, 0.0, 0.0, 0.0, 0.0, r*A_to_au_conversion])

    H, qubits = qchem.molecular_hamiltonian(
        symbols,
        geometry,
        active_electrons=active_electrons,
        active_orbitals=active_orbitals
    )

    active_electrons = active_electrons

    singles, doubles = qchem.excitations(active_electrons, qubits)

    print(f"Total number of excitations = {len(singles) + len(doubles)}")
    singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
    doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
    operator_pool = doubles_excitations + singles_excitations   
    hf_state = qchem.hf_state(active_electrons, qubits)
    dev = qml.device("default.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit():
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
        return qml.expval(H)
    energy_array = []
    opt = qml.optimize.AdaptiveOptimizer()
    for i in range(len(operator_pool)):
        circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
        energy_array.append(energy)
        if i % 3 == 0:
            print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
            # print(qml.draw(circuit, decimals=None)())
            print()
        if gradient < E*10^(-E):
            break
    return energy_array, circuit


### 3. Create Range

In [4]:
r_array = np.arange(0.5, 5.0, 0.25)
r_array

tensor([0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 , 2.75, 3.  ,
        3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75], requires_grad=True)

### 4. Calculate Energies 

In [5]:
E_R = [] # saved in E_R, contains E in each R
circuits = []

for i in range(len(r_array)):
    print(i)
    E, circuit = calc_adapt_vqe_LiH(r_array[i], 1)
    circuits.append(circuit)
    E_R.append(E[-1])

0
Total number of excitations = 24
n = 0,  E = -7.02840997 H, Largest Gradient = 0.108

